# Explore Valley Wind Events

Author: Daniel Hogan

Created: January 4, 2024

**Description:** This notebook seeks to establish when high wind events occurred within the East River valley in Colorado during the SAIL and SOS field campaigns. We will begin by looking at a specific event during the final days of 2022 and then gather some information about the atmospheric characteristics by looking at vertical wind profiles, radiosondes, and reanalysis data. We will then move on to a few other case studies from the analysis period to compare some of these different wind events and how valley orientation plays a role by starting to look at a few different wind observations around the region. 

### Imports

In [1]:
import xarray as xr 
from sublimpy import utils, variables
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px 
from act import discovery, plotting
import json
from scripts.get_sail_data import get_sail_data

nctoolkit is using Climate Data Operators version 2.3.0


# December 21 - December 27, 2022
## SOS Data Download

In [3]:
sos_5min_data = utils.download_sos_data(
                                        start_date='20221221',
                                        end_date='20221227',
                                        variable_names=variables.DEFAULT_VARIABLES,
                                        local_download_dir="../../01_data/raw_data/sosnoqc/",
                                        cache=True
                                    )                                  

Caching...skipping download for 20221221
Caching...skipping download for 20221222
Caching...skipping download for 20221223
Caching...skipping download for 20221224
Caching...skipping download for 20221225
Caching...skipping download for 20221226
Caching...skipping download for 20221227


## SAIL Data Download

Data was downloaded from the [ARM User Facility](https://www.arm.gov/research/campaigns/amf2021sail).
Variables downloaded include:

*Measured*

- Balloon-borne sounding system (BBSS): Vaisala-processed winds, pressure, temp, &RH (sondewnpn) released twice per day released from Gothic
- Surface meteorology at 10 m (met) located in Gothic
- Quality Controlled Eddy Correlation Flux Measurement - 30 min averaged (30qcecor) located at Kettle Ponds
- 915-MHz Radar Wind Profiler/RASS (RWP915): wind consensus data (915rwpwindcon) located in Gothic
- Doppler lidar wind value-added product (dlprofwind4news) located in Gothic
- Laser disdrometer (ld). One located in Gothic (M1) and the other located at Kettle Ponds (S2)

*Modeled*

- ECMWF: near-surface and surface variables (extra), entire coverage, 1-hr avg (ecmwfsfce)
- ECMWF: model multilevel surface fields at 4 levels, entire coverage (ecmwfsfcml)
- ECMWF: model met. and cloud variables at altitude, entire coverage, 1-hr avg (ecmwfvar)

In [4]:
# Function to load ARM credentials
def load_arm_credentials(credential_path):
    with open(credential_path, 'r') as f:
        credentials = json.load(f)
    return credentials
# Location of ARM credentials
credential_path = '/home/dlhogan/.act_config.json'
credentials = load_arm_credentials(credential_path)
# api token and username for ARM
api_username = credentials.get('username')
api_token = credentials.get('token')

In [19]:
# dictionary to store datastream names
sail_datastream_dict = {
    "radiosonde":"gucsondewnpnM1.b1",
    "met":"gucmetM1.b1",
    "eddy_covariance_kettle_ponds":"guc30qcecorS3.s1",
    "wind_profiler":"guc915rwpwindconM1.a1",
    "doppler_lidar":"gucdlprofwind4newsM1.c1",
    "laser_disdrometer_gothic":"gucldM1.b1",
    "laser_disdrometer_mt_cb":"gucldS2.b1",
    "ceilometer_10m_backscatter":"gucceil10mM1.b1",
}

In [23]:
# Set the location of the data folder where this data will be stored
event = 'dec2022'
sail_event_data_folder = f'/home/dlhogan/GitHub/Synoptic-Sublimation/01_data/raw_data/sail_{event}/'
# Set the start and end dates that we want to pull
startdate = '20221221'
enddate = '20221227'
# create empty data dictionary
sail_data_loc_dict = {}
# Iterate through the dictionary and pull the data for each datastream
for k,v in sail_datastream_dict.items():
    if (k =='radiosonde') & (len(os.listdir(os.path.join(sail_event_data_folder,"radiosonde"))) > 0):
        print("Radiosonde data donwloaded. Data files include:")
        # list file names in the radiosonde folder
        for file in os.listdir(os.path.join(sail_event_data_folder,"radiosonde")):
            print(file)
        print('-------------------')
    # Check if the file already exists
    elif (os.path.exists(f'{sail_event_data_folder}{k}_{startdate}_{enddate}.nc')): 
        print(f'{k}_{startdate}_{enddate}.nc already exists')
        print('-------------------')
        # add the filename to the dictionary which can be used if we want to load the data
        sail_data_loc_dict[k] = os.path.join(sail_event_data_folder,f'{k}_{startdate}_{enddate}.nc')
        continue
    else:
        # explicitly download radiosonde data because they are a lot easier to process and think about when in individual files
        if k == 'radiosonde':
            discovery.download_data(
                api_username,
                api_token,
                v,
                startdate=startdate,
                enddate=enddate,
                output=sail_event_data_folder+'radiosonde/'
            )
        else:
            ds = get_sail_data(api_username,
                        api_token,
                        v,
                        startdate=startdate,
                        enddate=enddate)
            ds.to_netcdf(f'{sail_event_data_folder}{k}_{startdate}_{enddate}.nc')
            sail_data_loc_dict[k] = os.path.join(sail_event_data_folder,f'{k}_{startdate}_{enddate}.nc')

Radiosonde data donwloaded. Data files include:
gucsondewnpnM1.b1.20221222.113600.cdf
gucsondewnpnM1.b1.20221223.112600.cdf
gucsondewnpnM1.b1.20221222.232700.cdf
gucsondewnpnM1.b1.20221223.233300.cdf
gucsondewnpnM1.b1.20221224.113000.cdf
gucsondewnpnM1.b1.20221224.232900.cdf
gucsondewnpnM1.b1.20221225.112600.cdf
gucsondewnpnM1.b1.20221225.232900.cdf
gucsondewnpnM1.b1.20221226.112800.cdf
gucsondewnpnM1.b1.20221226.232500.cdf
-------------------
met_20221221_20221227.nc already exists
-------------------
eddy_covariance_kettle_ponds_20221221_20221227.nc already exists
-------------------
wind_profiler_20221221_20221227.nc already exists
-------------------
doppler_lidar_20221221_20221227.nc already exists
-------------------
laser_disdrometer_gothic_20221221_20221227.nc already exists
-------------------
laser_disdrometer_mt_cb_20221221_20221227.nc already exists
-------------------
ceilometer_10m_backscatter_20221221_20221227.nc already exists
-------------------


In [25]:
# Load the met data from the sail dictionary into an xarray dataset
met_ds = xr.open_dataset(sail_data_loc_dict['met'])
sos_ds = xr.open_mfdataset("../../01_data/raw_data/sosnoqc/*.nc")
ecor_ds = xr.open_dataset(sail_data_loc_dict['eddy_covariance_kettle_ponds'])